In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
# !pip install openpyxl
# https://github.com/maryami66/uci_dataset

# https://archive-beta.ics.uci.edu/datasets?Types=Text&Task=Regression&sort=desc&orderBy=NumHits&skip=0&take=10
# 

In [7]:
data.head()
#data['name'] = data['name'].str.lower()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [8]:
tfidf = TfidfVectorizer().fit(data['name'])

NameError: name 'TfidfVectorizer' is not defined

In [9]:
tfidf.transform(data['name']).todense()[:5, tfidf.get_feature_names_out() == 'park']

NameError: name 'tfidf' is not defined

In [10]:
data = pd.read_csv('data/AB_NYC_2019.csv')

TARGET = ['price']
CATEGORICAL = ['neighbourhood_group', 'neighbourhood', 'room_type']
FEATURES = ['name', 'minimum_nights'] + CATEGORICAL
data['name'] = data['name'].fillna('')
X, y = data[FEATURES], data[TARGET]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [12]:
# https://www.kaggle.com/code/prashantsaraswat88/using-text-based-features-for-price-regression

## Count Vecorizer

In [13]:
#from xgboost import XGBRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import mean_squared_error

from sklearn.compose import ColumnTransformer, make_column_selector
from category_encoders import OneHotEncoder

from sklearn.linear_model import LinearRegression

# Count vectorizer
count_vec_pipeline = Pipeline([
    ('one_hot', OneHotEncoder(cols=CATEGORICAL)),
    ('count_vectorizer', ColumnTransformer([
        ('name_cv', CountVectorizer(), 'name')
    ])),
    ('linear_regression', LinearRegression())
])
count_vec_pipeline.fit(X_train, y_train)
y_pred_count_vec = count_vec_pipeline.predict(X_test)
print(mean_squared_error(y_test, y_pred_count_vec))

# # TFIDF
tfidf_pipeline = Pipeline([
    ('one_hot', OneHotEncoder(cols=CATEGORICAL)),
    ('count_vectorizer', ColumnTransformer([
        ('name_tfidf', TfidfVectorizer(), 'name')
    ])),
    ('linear_regression', LinearRegression())
])
tfidf_pipeline.fit(X_train, y_train)
y_pred_tfidf = tfidf_pipeline.predict(X_test)
print(mean_squared_error(y_test, y_pred_tfidf))

58399.326446735955
66950.35023982753


In [14]:
feature_names = tfidf_pipeline[1].get_feature_names_out()
tfidf_features = pd.DataFrame(
    tfidf_pipeline[1].transform(X_test).todense(), 
    columns=feature_names
)
tfidf_features.loc[:, np.random.choice(tfidf_features.columns, 20)]

,name_tfidf__sprout,name_tfidf__petit,name_tfidf__chris,name_tfidf__1899,name_tfidf__chelsa,name_tfidf__駅徒歩5分の好立地なお部屋です,name_tfidf__apatite,name_tfidf__price,name_tfidf__sweat,name_tfidf__magnifique,name_tfidf__stars,name_tfidf__flowery,name_tfidf__18,name_tfidf__southslope,name_tfidf__sterling,name_tfidf__chillin,name_tfidf__penthouse_private,name_tfidf__哥伦比亚大学附近步行3分钟高档公寓主卧暑期降价转租,name_tfidf__manatthan,name_tfidf__cobblestone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
feature_names

array(['name_tfidf__00', 'name_tfidf__000', 'name_tfidf__000sf', ...,
       'name_tfidf__ﾏﾝﾊｯﾀﾝ', 'name_tfidf__ﾏﾝﾊｯﾀﾝ10分',
       'name_tfidf__ﾐｯﾄﾞﾀｳﾝｳｴｽﾄｻｲﾄﾞ'], dtype=object)

In [16]:
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation

# Topic modelling

In [17]:
# Count vectorizer
tm_pipeline = Pipeline([
    ('one_hot', OneHotEncoder(cols=CATEGORICAL)),
    ('count_vectorizer', ColumnTransformer([
        ('name', make_pipeline(
            CountVectorizer(stop_words='english'), 
            NMF(n_components=10, random_state=7)), 'name')
    ])),
    ('linear_regression', LinearRegression())
])
tm_pipeline.fit(X_train, y_train)
y_pred_count_vec = tm_pipeline.predict(X_test)
print(mean_squared_error(y_test, y_pred_count_vec))

48795.543308842796


## Cross validation

In [18]:
from sklearn.model_selection import cross_val_score

# scores = cross_val_score(tm_pipeline, X_train, y_train, cv=5, scoring=score_fun)

In [19]:
import optuna
from sklearn.model_selection import cross_val_score

def score_fun(est, X, y):
    return mean_squared_error(y, est.predict(X))


def objective(trial):
    n_components = trial.suggest_int('n_components', 10, 100)
    
    tm_pipeline = Pipeline([
        ('one_hot', OneHotEncoder(cols=CATEGORICAL)),
        ('count_vectorizer', ColumnTransformer([
            ('name', make_pipeline(
                CountVectorizer(stop_words='english'), 
                TruncatedSVD(n_components=n_components, random_state=7)), 'name')
        ])),
        ('linear_regression', LinearRegression())
    ])
    
    scores = cross_val_score(tm_pipeline, X_train, y_train, cv=5, scoring=score_fun)
    return np.mean(scores)

study = optuna.create_study()
study.optimize(objective, n_trials=5)

[I 2023-06-06 23:35:07,440] A new study created in memory with name: no-name-56880ece-3c9a-409b-b853-7afdfd567741
[I 2023-06-06 23:35:11,294] Trial 0 finished with value: 57148.534929045396 and parameters: {'n_components': 18}. Best is trial 0 with value: 57148.534929045396.
[I 2023-06-06 23:35:15,000] Trial 1 finished with value: 57327.43310948989 and parameters: {'n_components': 15}. Best is trial 0 with value: 57148.534929045396.
[I 2023-06-06 23:35:18,762] Trial 2 finished with value: 57430.62598849076 and parameters: {'n_components': 11}. Best is trial 0 with value: 57148.534929045396.
[I 2023-06-06 23:35:23,616] Trial 3 finished with value: 56419.028132129984 and parameters: {'n_components': 33}. Best is trial 3 with value: 56419.028132129984.
[I 2023-06-06 23:35:33,448] Trial 4 finished with value: 55453.29832632545 and parameters: {'n_components': 85}. Best is trial 4 with value: 55453.29832632545.


In [20]:
study.best_params

{'n_components': 85}

In [21]:
cv.fit(X_train['name']).get_feature_names_out()

NameError: name 'cv' is not defined

In [ ]:
tm_pipeline[1].transform(X[:5])[[0, 3, 4], :4]

In [ ]:
count_vectorizer = tm_pipeline[1].transformers_[0][1][0]
decomposition = tm_pipeline[1].transformers_[0][1][1]

In [ ]:
import matplotlib.pyplot as plt

def plot_top_words(model, feature_names, n_top_words, title, n_topics=10):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_[:n_topics]):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
plot_top_words(
    decomposition,
    count_vectorizer.get_feature_names_out(),
    10,
    "Topics in CountVectorizer + NMF",
)

In [ ]:
tm_pipeline[1].transform(X).argmax(axis=1)

In [ ]:
result = tm_pipeline[1].transform(X)

In [ ]:
full_encoder  = tm_pipeline[1]
bag_of_words  = full_encoder.transformers_[0][1][0]
decomposition = full_encoder.transformers_[0][1][1]

In [ ]:
def docs_vs_topics(full_encoder, bag_of_words, decomposition, 
                   n_docs=20, n_top_words=5):
    # Based on https://dirty-cat.github.io/stable/auto_examples/02_investigating_dirty_categories.html#sphx-glr-auto-examples-02-investigating-dirty-categories-py
    
    # Exctracting topic labels
    feature_names = bag_of_words.get_feature_names_out()
    out_features = full_encoder.get_feature_names_out()
    topic_labels = []
    
    for k in range(len(out_features)):
        labels = out_features[k]       
        topic = decomposition.components_[k]
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        topic_labels.append(','.join(top_features))
    
    # Plotting matrix
    encoded_labels = full_encoder.transform(X[:n_docs])
    plt.figure(figsize=(8, len(out_features)))
    plt.imshow(encoded_labels)
    plt.xlabel("Latent topics", size=12)
    plt.xticks(range(0, len(out_features)), labels=topic_labels, rotation=50, ha="right")
    plt.ylabel("Documents", size=12)
    plt.yticks(range(0, n_docs), labels=X['name'][:n_docs].to_numpy().flatten())
    plt.colorbar().set_label(label="Topic activations", size=12)
    plt.tight_layout()
    plt.show()

docs_vs_topics(
    full_encoder  = full_encoder, 
    bag_of_words  = bag_of_words,
    decomposition = decomposition,
    n_docs        = 30
)

In [ ]:
decomposition

## Hashing

In [22]:
from sklearn.feature_extraction.text import HashingVectorizer

# Count vectorizer
hash_vec_pipeline = Pipeline([
    ('one_hot', OneHotEncoder(cols=CATEGORICAL)),
    ('count_vectorizer', ColumnTransformer([
        ('hash_vec', HashingVectorizer(), 'name')
    ])),
    ('linear_regression', LinearRegression())
])
hash_vec_pipeline.fit(X_train, y_train)
y_pred_count_vec = hash_vec_pipeline.predict(X_test)
print(mean_squared_error(y_test, y_pred_count_vec))

72355.40732227267


In [78]:
hash_vec = hash_vec_pipeline[1].transformers_[0][1]
tokenizer = hash_vec.build_analyzer()
hasher = hash_vec._get_hasher()

In [105]:
tokenized = [tokenizer(string) for string in X_train['name']]
unique_tokens = np.unique([token for lst in tokenized for token in lst])

In [106]:
hashes = hasher.transform(np.array([unique_tokens]).T)

In [107]:
token_indices = np.abs(hashes).argmax(axis=1)

In [115]:
token_dict = pd.DataFrame({
    'token': np.array(unique_tokens).ravel().tolist(),
    'idx': token_indices.ravel().tolist()[0]  
})

In [116]:
token_dict

,token,idx
0,00,543187
1,000,494718
2,000sf,312141
3,000sqft,943828
4,001,591712
...,...,...
7146,ｷﾚｲなお部屋,27204
7147,ﾆｭｰﾖｰｸの中心,345399
7148,ﾏﾝﾊｯﾀﾝ,634584
7149,ﾏﾝﾊｯﾀﾝ10分,1042381


In [128]:
hash_collision = token_dict.groupby('idx').count()


idx
150        False
251        False
465        False
638        False
646        False
           ...  
1047950    False
1048013    False
1048038    False
1048270    False
1048305    False
Name: token, Length: 7131, dtype: bool

In [136]:
colliding_hashes = hash_collision[(hash_collision > 1).token].reset_index().idx

In [137]:
colliding_hashes

0      67599
1      68490
2      70887
3     181264
4     188967
5     255613
6     256983
7     342197
8     454853
9     658585
10    723532
11    730602
12    765899
13    783375
14    823969
15    833352
16    870506
17    904887
18    980867
19    991869
Name: idx, dtype: int64

In [141]:
same_hash = token_dict.query('idx in @colliding_hashes').sort_values('idx')

In [142]:
same_hash

,token,idx
3593,immaculate,67599
3543,hrlem,67599
6485,uppereast,68490
3132,futon,68490
4112,luxury,70887
4288,med,70887
6224,thompson,181264
2720,eden,181264
4239,marlie,188967
4191,manhattan20,188967
